In [ ]:
import numpy as np

def process_obj_and_inverse_y(input_obj_path, output_obj_path, center_object=True):
    """
    Lit un fichier OBJ avec des sommets (et potentiellement des couleurs),
    inverse l'objet selon l'axe Y, et peut aussi le centrer.
    Le résultat est sauvegardé dans un nouveau fichier OBJ en préservant les informations de couleur.

    Args:
        input_obj_path (str): Chemin du fichier OBJ d'entrée.
        output_obj_path (str): Chemin du fichier OBJ de sortie.
        center_object (bool): Si True, l'objet sera centré avant l'inversion Y.
                              (Note: le centrage est appliqué après l'inversion Y pour une cohérence
                              avec l'exemple précédent, mais l'ordre peut varier selon le besoin précis).
    """
    vertices = []
    vertex_colors = []
    faces = []
    normals = []
    texcoords = []

    print(f"Lecture du fichier OBJ : {input_obj_path}")
    with open(input_obj_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue

            if parts[0] == 'v':
                v = [float(p) for p in parts[1:4]]
                vertices.append(v)
                if len(parts) == 7:
                    c = [float(p) for p in parts[4:7]]
                    vertex_colors.append(c)
                else:
                    vertex_colors.append(None)
            elif parts[0] == 'vn':
                normals.append([float(p) for p in parts[1:4]])
            elif parts[0] == 'vt':
                texcoords.append([float(p) for p in parts[1:3]])
            elif parts[0] == 'f':
                faces.append(parts[1:])

    if not vertices:
        print("Aucun sommet trouvé dans le fichier OBJ. Opération annulée.")
        return

    vertices_np = np.array(vertices)

    # --- Étape de centrage (optionnelle) ---
    if center_object:
        min_coords = np.min(vertices_np, axis=0)
        max_coords = np.max(vertices_np, axis=0)
        center = (min_coords + max_coords) / 2.0
        print(f"Centrage de l'objet par rapport à : {center}")
        vertices_np = vertices_np - center

    # --- Inversion selon l'axe Y ---
    # Multipliez la composante Y (deuxième colonne, index 1) par -1
    vertices_np[:, 1] *= -1
    print("Inversion des sommets selon l'axe Y appliquée.")

    print(f"Sauvegarde de l'objet transformé dans : {output_obj_path}")
    with open(output_obj_path, 'w') as f:
        for i, v_transformed in enumerate(vertices_np):
            f.write(f"v {v_transformed[0]:.6f} {v_transformed[1]:.6f} {v_transformed[2]:.6f}")
            if vertex_colors[i] is not None:
                c = vertex_colors[i]
                f.write(f" {c[0]:.6f} {c[1]:.6f} {c[2]:.6f}")
            f.write("\n")

        # Écrire les normales (Note: les normales devraient idéalement être transformées aussi
        # pour refléter l'inversion, mais pour une inversion simple de l'axe Y des sommets,
        # nous les laissons telles quelles si ce n'est pas nécessaire pour votre usage).
        # Pour une inversion correcte des normales sur Y, il faudrait faire : normals_np[:, 1] *= -1
        for vn in normals:
            f.write(f"vn {vn[0]:.6f} {vn[1]:.6f} {vn[2]:.6f}\n")

        for vt in texcoords:
            f.write(f"vt {vt[0]:.6f} {vt[1]:.6f}\n")

        # Écrire les faces
        for face in faces:
            f.write("f " + " ".join(face) + "\n")

    print("Traitement terminé avec succès.")

In [ ]:
input_file = "../graphics/visapp/3d/gorgoile/sommets_visibles_pose_i7-j2.obj"
output_file = input_file.replace('_pose_i7-j2.obj', '_centered.obj')

process_obj_and_inverse_y(input_file, output_file_centered_and_y_inverted, center_object=True)

# Vous pouvez inspecter 'output_centered.obj' pour voir les sommets centrés
# et les couleurs préservées.

Lecture du fichier OBJ : ../graphics/visapp/3d/gorgoile/sommets_visibles_pose_i7-j2.obj
Centre de l'objet détecté : [ 0.09768891 -0.00851461  2.29176988]
Sauvegarde de l'objet centré dans : ../graphics/visapp/3d/gorgoile/sommets_visibles_centered.obj
Traitement terminé avec succès.
